<img src="images/header.png">

<h1><center>Алгоритмы интеллектуальной обработки больших объемов данных</center></h1>
<hr>
<h2><center>Линейные модели (практика 2 - Vowpal Wabbit)</center></h2>

<img src=https://cdn.dribbble.com/users/261617/screenshots/3146111/vw-dribbble_1x.png>

[Vowpal Wabbit](https://github.com/JohnLangford/vowpal_wabbit/wiki) - это библиотека с большим набором оптимизированных итеративных алгоритмов машинного обучения

В этом туториале мы применим эту библиотеку с целью решить задачу классификации с помощью логистической регрессии.

**Туториалы:**
https://github.com/JohnLangford/vowpal_wabbit/wiki/Tutorial

**Туториал для сборки в unix:**
https://github.com/JohnLangford/vowpal_wabbit/wiki/v7.0_tutorial.pdf

# Загрузка и подготовка данных

https://www.kaggle.com/dalpozz/creditcardfraud/data

In [ ]:
import requests
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)


In [ ]:
fn = 'creditcard.csv.zip'
fn_out_train = 'creditcard_vw_format_train'
fn_out_test = 'creditcard_vw_format_test'
fn_out_validation = 'creditcard_vw_format_validation'

fn_out_train_weight = 'creditcard_vw_format_train_weight'
fn_out_test_weight = 'creditcard_vw_format_test_weight'
fn_out_validation_weight = 'creditcard_vw_format_validation_weight'

In [ ]:
df = pd.read_csv(fn, compression='zip', header=0, sep=',', quotechar='"')

In [ ]:
df['Class'] = df['Class']* 2 - 1

In [ ]:
df.head()

In [ ]:
df['Class'].value_counts()

In [ ]:
df.info()

In [ ]:
df.describe()

**Time** - the seconds elapsed between each transaction and the first transaction in the dataset. Numeric  
**V1** - First principle component Numeric  
**V2** - Second principle component Numeric  
**V3** - Third principle component Numeric  
...  
**V28** - Twenty-eighth principle component Numeric  
**Amount** - Transaction Amount Numeric   
**Class** - The actual classification classes Numeric  

The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available on http://mlg.ulb.ac.be/BruFence and http://mlg.ulb.ac.be/ARTML

Please cite: Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

# Формат данных

[Описание формата](https://github.com/JohnLangford/vowpal_wabbit/wiki/Input-format)


Например:

1 1.0 zebra|MetricFeatures:3.28 height:1.5 length:2.0 |Says black with white stripes |OtherFeatures NumberOfLegs:4.0 HasStripes

Приведем данные к этому формату

In [ ]:
skip_columns = {'Class', 'Index', 'Time'}

def df_to_file(file_name, data):
    with open(file_name, 'w') as f:
        for row in tqdm_notebook(data.itertuples()):
            features = row._asdict()
            features_formatted = ' '.join('{}:{}'.format(k, v) for k, v in features.iteritems() if k not in skip_columns)
            f.write('{} | {}\n'.format(row.Class, features_formatted))

In [ ]:
train, test = train_test_split(df, test_size=0.2)
train, validation = train_test_split(train, test_size=0.2)

In [ ]:
df_to_file(fn_out_train, train)
df_to_file(fn_out_validation, validation)

In [ ]:
%%sh -s "$fn_out_train" "$fn_out_validation" "$fn_out_test"

wc -l $1
head $1
wc -l $2

# Команды

[Описание аргументов](https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments)

## Обучение

В vw реализован модифицированный градиентный спуск: http://www.cs.cmu.edu/~sross1/publications/uai13_normalized.pdf

In [ ]:
%%sh -s "$fn_out_train" "$fn_out_validation" "$fn_out_test"

vw -d $1 -f ./simple_model.vw \
--sgd --adaptive --normalized --invariant \
--loss_function=logistic -l 0.5  --decay_learning_rate 0.95 \
--passes 4 --holdout_off --cache_file ./data.cache

rm ./data.cache

## Предсказание

In [ ]:
%%sh -s "$fn_out_train" "$fn_out_validation" "$fn_out_test"

vw -i ./simple_model.vw -t -d $2 -p ./simple_model_predictions.vw --link=logistic \
--invert_hash ./simple_model.vw.read

In [ ]:
!cat ./simple_model.vw.read

In [ ]:
!head simple_model_predictions.vw

## Оценка качества

In [ ]:
with open('simple_model_predictions.vw') as pred_file:
    validation_prediction = [float(label) for label in pred_file.readlines()]

auc = roc_auc_score(validation['Class'], validation_prediction)
curve = roc_curve(validation['Class'], validation_prediction)

plt.plot(curve[0], curve[1]);
plt.plot([0,1], [0,1])
plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('test AUC = %f' % (auc)); plt.axis([-0.05,1.05,-0.05,1.05]);

## Взвешивание объектов

Добавим вес меньшему классу

In [ ]:
from sklearn.utils import compute_class_weight

C помощью фунции `compute_class_weight` посчитайте веса классов и напишите функцию `df_to_file_weight`

In [ ]:
df_to_file_weight(fn_out_train_weight, train)
df_to_file_weight(fn_out_validation_weight, validation)

In [ ]:
%%sh -s "$fn_out_train_weight" "$fn_out_validation_weight" "$fn_out_test_weight"

vw -d $1 -f ./model_weights.vw \
--sgd --adaptive --normalized --invariant \
--loss_function=logistic -l 0.9 --decay_learning_rate 0.95 \
--passes 4 --holdout_off --cache_file ./data.cache

rm ./data.cache

In [ ]:
%%sh -s "$fn_out_train_weight" "$fn_out_validation_weight" "$fn_out_test_weight"

vw -i model_weights.vw  -t -d $2 -p model_weights_predictions.vw --link=logistic \
--invert_hash ./model_weights.vw.read

In [ ]:
!cat model_weights.vw.read

In [ ]:
with open('model_weights_predictions.vw') as pred_file:
    validation_prediction = [float(label) for label in pred_file.readlines()]

auc = roc_auc_score(validation['Class'], validation_prediction)
curve = roc_curve(validation['Class'], validation_prediction)

plt.plot(curve[0], curve[1]);
plt.plot([0,1], [0,1])
plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('test AUC = %f' % (auc)); plt.axis([-0.05,1.05,-0.05,1.05]);

Исследуйте другие "фичи", например Stagewise Polynomial, Quadratic features, --nn, --boosting...